# IMPORT

In [167]:
pip install pycryptodome

In [168]:
from Crypto.Cipher import AES
from math import ceil

# FUNCTION


In [169]:
def convert_hex_str_to_int_list(string):
    return [int(string[i: i+2], 16) for i in range(0, len(string), 2)]

'''
Function to convert a hex_string -> list of int (each 2 bytes)
'''

'\nFunction to convert a hex_string -> list of int (each 2 bytes)\n'

### CBC

In [170]:
def decrypt_cbc(key, cipher):
    
    key_list = convert_hex_str_to_int_list(key) # string -> list of int
    cipher_list_contain_iv = convert_hex_str_to_int_list(cipher) # IV | y0 | y1 | ... | yn
    block_size = len(key_list) # size of 1 block 
    number_of_blocks = ceil(len(cipher_list_contain_iv)/len(key_list)) # number of cipher blocks
    aes_decoder = AES.new(bytes(key_list), AES.MODE_ECB) # aes decoder
    iv_list, cipher_list = cipher_list_contain_iv[:block_size], cipher_list_contain_iv[block_size:] # extract iv and cipher from initial cipher_list IV | y0 | y1 | ... | yn
    iv = bytes(iv_list) # -> stringbytes

    message_list = [] # array to append message blocks after loop
    for i in range(number_of_blocks): # loop to calculate each block of plaintext
        dec = aes_decoder.decrypt(bytes(cipher_list[i*block_size : (i+1)*block_size])) # dec = DEC(key, cipher)
        message = [iv[j] ^ dec[j] for j in range(len(dec))] # message = iv XOR DEC(key, cipher)
        message_list.extend(message) # add block of plaintext [m0, m1, ...]
        iv = bytes(cipher_list[i*block_size : (i+1)*block_size]) # set new iv for next iteration

    return "".join([chr(x) for x in message_list[: len(message_list) - message_list[len(message_list)-1]]]) # [m0, m1, ...] -> message


In [171]:
def encrypt_cbc(key, plaintext, iv):
    key_list = convert_hex_str_to_int_list(key) # string -> list of int
    plaintext_list = convert_hex_str_to_int_list(key) #  m0 | m1 | ... | mn
    block_size = len(key_list) # size of 1 block
    number_of_blocks = ceil(len(plaintext_list)/len((key_list))) # numbers of block
    plaintext_list = padding(plaintext_list, block_size) # padding step
    aes_decoder = AES.new(bytes(key_list), AES.MODE_ECB) # aes decoder
    iv = convert_hex_str_to_int_list(iv) # iv string -> list of int

    cipher_list = [] # array to append message
    cipher_list.extend(iv) # cipher_list = iv | .. | .. | ..
    for i in range(number_of_blocks): # loop to encrypt each block
        plain = plaintext_list[i*block_size : (i+1)*block_size] # get block plaintext at step i
        before_aes_encrypt = [iv[j] ^ plain[j] for j in range(len(block_size))] # before_aes_encrypt = iv XOR message
        cipher = aes_decoder.encrypt(bytes(before_aes_encrypt)) # cipher = ENC(key, iv XOR m)
        iv = cipher # new iv = cipher = ENC(key, iv XOR m)
        cipher_list.extend(cipher) # add block of ciphertext iv | c0 | c1 | ...
    return "".join([chr(x) for x in cipher_list]) # [iv, c0, c1, ...] -> ciphertext


### CTR

In [172]:
def decrypt_ctr(key, cipher):

    key_list = convert_hex_str_to_int_list(key) # string -> list of int
    cipher_list_contain_iv = convert_hex_str_to_int_list(cipher) # IV | y0 | y1 | ... | yn
    block_size = len(key_list) # size of 1 block
    iv_list, cipher_list = cipher_list_contain_iv[:block_size], cipher_list_contain_iv[block_size:] # extract iv and cipher from initial cipher_list IV | y0 | y1 | ... | yn
    number_of_blocks = ceil(len(cipher_list_contain_iv)/len(key_list)) # number of cipher blocks
    
  
    aes_decoder = AES.new(bytes(key_list), AES.MODE_ECB) # aes decoder
    message_list = [] # array to append message blocks
    for i in range(number_of_blocks): # loop to calculate each plaintext block
        iv = iv_step(iv_list[:], i) # iv = iv + i
        dec = aes_decoder.encrypt(bytes(iv)) # dec = ENC(key, iv)
        cipher = cipher_list[i*block_size : (i+1)*block_size] # ciphertext block in step i
        message = [dec[j] ^ cipher[j] for j in range(len(cipher))] # message = ENC(key, iv) XOR ciphertext
        message_list.extend(message) # add block of plaintext [m0, m1, ...]
    return "".join([chr(x) for x in message_list]) # [m0, m1, ...] -> message

In [173]:
def encrypt_ctr(key, plaintext, iv):

    key_list = convert_hex_str_to_int_list(key) # string -> list of int
    plaintext_list = convert_hex_str_to_int_list(plaintext) #  m0 | m1 | ... | mn
    block_size = len(key_list) # size of 1 block 
    number_of_blocks = ceil(len(cipher_list_contain_iv)) # number of block in ciphertext
    plaintext_list = padding(plaintext_list) # padding step
    aes_decoder = AES.new(bytes(key_list), AES.MODE_ECB) # aes decoder
  
    cipher_list = [] # array to append message blocks
    cipher_list.extend(iv) # ciphertext = iv | .. | .. |
    for i in range(number_of_blocks): # loop to encrypt each block
        iv = iv_step(iv_list, i) # iv = iv + 1
        dec = aes_decoder.encrypt(bytes(iv)) # dec = ENC(key, iv)
        plain = plain_list[i*block_size : (i+1)*block_size] # get plaintext block in step i
        cipher = [dec[j] ^ plain[j] for j in range(len(plain))] # cipher = ENC(key, iv) XOR m 
        cipher_list.extend(cipher) # add block of cipher iv | c0 | c1 | ...
    return "".join([chr(x) for x in cipher_list]) # [iv, c0, c1, ...] -> ciphertext

In [174]:
def iv_step(iv_list, step): # get iv + i for each step
    iv_size = len(iv_list) # size of 1 block

    # loop for calculating iv + 1
    for i in range(iv_size - 1, -1, -1): 
        j = iv_list[i] + step
        if j % 0xFF == j:
            iv_list[i] = j
            break
        else:
            iv_list[i] = j % 0xFF
            step = int(j/0xFF)
    return iv_list


### Padding - Unpadding PCKS5

In [175]:
def padding(plaintext_list, block_size):
    plaintext_size = len(plaintext_list) # size of 1 block
    if len(plaintext_list) % block_size == 0: # add dummy blocks
        for i in range(block_size):
            plaintext_list.extend(blocksize)
    else: # add b block b (PKCS5)
        remainder = len(plaintext_list) % block_size # remainder is excess part
        need_to_add = block_size - remainder # remainder + need_to_add = size of 1 block
        for i in range(need_to_add): # loop to add b block b
            plaintext_list.extend(need_to_add)
    return plaintext_list

In [176]:
def unpadding(plaintext_list, block_size):
    number_of_blocks = len(plaintext_list)/block_size # numbers of block
    last_block = plaintext_list[(number_of_blocks - 1)*block_size : number_of_blocks*block_size] # get last block for unpadding
    if last_block[-1] == block_size: # remove all if it's a dummy block
        plaintext_list = plaintext_list[:(number_of_blocks - 1)*block_size]
        return plain_text
    padding = last_block[-1] # get infor of last bytes
    plaintext_list = plaintext_list[:number_of_blocks*block_size - padding] # remove number of bytes depend on the last bytes (PKCS5)
    return plaintext_list 

# RUN MAIN


In [177]:
if __name__ == "__main__":

    # load keys and ciphertexts

    key1 = key2 = '140b41b22a29beb4061bda66b6747e14'
    key3 = key4 = '36f18357be4dbd77f050515c73fcf9f2'
    cp1 = '4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81'
    cp2 = '5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253'
    cp3 = '69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329'
    cp4 = '770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451'

    print("Plaintext1: ", decrypt_cbc(key1, cp1))
    print("Plaintext2: ", decrypt_cbc(key2, cp2))
    print("Plaintext3: ", decrypt_ctr(key3, cp3))
    print("Plaintext4: ", decrypt_ctr(key4, cp4))

Plaintext1:  Basic CBC mode encryption needs padding.
Plaintext2:  Our implementation uses rand. IV
Plaintext3:  CTR mode lets you build a stream cipher from a block cipher.
Plaintext4:  Always avoid the two time pad!
